# Milestone 2：流程演示（预处理/ICA/事件/Epochs/ERP）

展示我们与作者不同但合理的流程：平均参考 + 0.1–30 Hz + 50 Hz 隔离 + ICA（可选 ICLabel）+ 合并反馈事件 + Epochs（-0.2~0.8 s）。FCz 处绘制各条件 ERP 与差波。

In [ ]:
# 路径设置
import sys
from pathlib import Path
ROOT = Path.cwd()
while ROOT != ROOT.parent and not (ROOT / 'src').exists():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))
print('Repo root:', ROOT)

In [ ]:
# 导入依赖与项目函数
import mne, numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
from src.config import ensure_dirs, FIGURES_DIR, RESULTS_DIR
from src.preprocessing import read_raw_bids, PreprocParams, preprocess_raw, fit_ica
from src.events import get_feedback_events
from src.erp import make_epochs, compute_evokeds_by_condition
SUBJECT = '28'
ensure_dirs()

In [ ]:
# 1) 预处理（ERP 参数）
params = PreprocParams(l_freq=0.1, h_freq=30.0, notch=50.0, use_icalabel=True)
raw = read_raw_bids(SUBJECT)
raw_p = preprocess_raw(raw, params)

# 连续片段图
fig_cont = raw_p.plot(duration=10.0, n_channels=32, show=False)
p_cont = FIGURES_DIR / f'm2_continuous_{SUBJECT}.png'
fig_cont.savefig(p_cont, dpi=150)
display(Image(filename=str(p_cont)))

# ICA（若可用将自动标注；否则仅拟合）
ica = fit_ica(raw_p, params)
raw_ica = ica.apply(raw_p.copy())
fig_ica = ica.plot_components(show=False)
p_ica = FIGURES_DIR / f'm2_ica_components_{SUBJECT}.png'
fig_ica.savefig(p_ica, dpi=150)
display(Image(filename=str(p_ica)))

# PSD 概览
fig_psd = raw_p.plot_psd(show=False)
p_psd = FIGURES_DIR / f'm2_psd_{SUBJECT}.png'
fig_psd.savefig(p_psd, dpi=150)
display(Image(filename=str(p_psd)))

In [ ]:
# 2) 反馈事件与 Epochs（-0.2~0.8 s）
events, event_id = get_feedback_events(raw)
fig_ev = mne.viz.plot_events(events, event_id=event_id, show=False)
p_events = FIGURES_DIR / f'm2_events_{SUBJECT}.png'
fig_ev.savefig(p_events, dpi=150)
display(Image(filename=str(p_events)))

epochs = make_epochs(raw_ica, events, event_id, tmin=-0.2, tmax=0.8, baseline=None)
conds = ['high_win','high_loss','mid_win','mid_loss','low_win','low_loss']
evokeds = compute_evokeds_by_condition(epochs, conds)

In [ ]:
# 3) FCz ERP 图（叠加）
plt.figure(figsize=(8,4))
for k in conds:
    if k in evokeds:
        y = evokeds[k].copy().pick('FCz').data[0] * 1e6
        t = evokeds[k].times
        plt.plot(t, y, label=k)
plt.axvline(0, color='k', linestyle='--', linewidth=1)
plt.axhline(0, color='gray', linestyle='-', linewidth=0.5)
plt.xlim(-0.2, 0.8)
plt.xlabel('时间 (s)')
plt.ylabel('电位 (μV) @ FCz')
plt.title('条件 ERP（FCz）')
plt.legend(loc='best', fontsize=8)
plt.tight_layout()
p_evoked = FIGURES_DIR / f'm2_evoked_fcz_{SUBJECT}.png'
plt.savefig(p_evoked, dpi=150)
display(Image(filename=str(p_evoked)))

# 4) 写入简要报告
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
report = RESULTS_DIR / f'm2_summary_{SUBJECT}.txt'
with open(report, 'w', encoding='utf-8') as f:
    f.write(f'Subject: {SUBJECT}\n')
    f.write('Conditions present in epochs: \n+')
    for k in evokeds.keys():
        f.write(f'  {k}: n={len(epochs[k])}\n')
    for p in [p_cont, p_psd, p_ica, p_events, p_evoked]:
        f.write(f'  FIG: {p}\n')
print('Saved report:', report)